In [130]:
%%writefile Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
SHELL ["/bin/bash", "-c"]
RUN apt-get update -y && apt-get -y install kubectl
COPY get_data.py .
COPY split_data.py .
COPY data_Transformation.py .
COPY model_training.py .
COPY Train_set_data_transformation.py .
COPY evaluation.py .
COPY requirements.txt .
RUN python -m pip install -U -r requirements.txt 

Overwriting Dockerfile


In [131]:
IMAGE_NAME="train_image"
TAG="latest"
IMAGE_URI="gcr.io/ai-platform-01/{}:{}".format(IMAGE_NAME,TAG)
print(IMAGE_URI)

gcr.io/ai-platform-01/train_image:latest


In [132]:
!gcloud builds submit --timeout 15m --tag {IMAGE_URI} 

Creating temporary tarball archive of 35 file(s) totalling 5.7 MiB before compression.
Uploading tarball of [.] to [gs://ai-platform-01_cloudbuild/source/1635344896.771233-33cf21766a444541b6e86b896adaca06.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/ai-platform-01/locations/global/builds/0bead6cd-4497-467a-a70b-3f60a519944d].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/0bead6cd-4497-467a-a70b-3f60a519944d?project=659323410291].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "0bead6cd-4497-467a-a70b-3f60a519944d"

FETCHSOURCE
Fetching storage object: gs://ai-platform-01_cloudbuild/source/1635344896.771233-33cf21766a444541b6e86b896adaca06.tgz#1635344898492226
Copying gs://ai-platform-01_cloudbuild/source/1635344896.771233-33cf21766a444541b6e86b896adaca06.tgz#1635344898492226...
/ [1 files][  1.3 MiB/  1.3 MiB]                                                
Operation completed over 1 objects/1.

In [ ]:
!pip install kfp

In [95]:
import kfp
import kfp.components as comp
from kubernetes.client.models import V1EnvVar

In [96]:
@kfp.dsl.component
def get_data():
    # Defining component configuration
    getdata_component = kfp.dsl.ContainerOp(
        name='Downloading data',
        image='gcr.io/ai-platform-01/train_image:latest',
        command=['python', 'get_data.py'],
        )
    return getdata_component

In [97]:
@kfp.dsl.component
def split():
    # Defining component configuration
    split_component = kfp.dsl.ContainerOp(
        name='Train and Test Split',
        image='gcr.io/ai-platform-01/train_image:latest',
        command=['python', 'split_data.py'],
        )
    return split_component

In [98]:
@kfp.dsl.component
def data_transformation():
    # Defining component configuration
    transformation_component = kfp.dsl.ContainerOp(
        name='data transformation',
        image='gcr.io/ai-platform-01/train_image:latest',
        command=['python', 'data_Transformation.py'],
        )
    return transformation_component

In [99]:
@kfp.dsl.component
def model_train():
    # Defining component configuration
    train_component = kfp.dsl.ContainerOp(
        name='Model Training',
        image='gcr.io/ai-platform-01/train_image:latest',
        command=['python', 'model_training.py'],
        )
    return train_component

In [100]:
@kfp.dsl.component
def test_transformation():
    # Defining component configuration
    test_data_transformation_component = kfp.dsl.ContainerOp(
        name='Test data transformation',
        image='gcr.io/ai-platform-01/train_image:latest',
        command=['python', 'Train_set_data_transformation.py'],
        )
    return test_data_transformation_component

In [133]:
@kfp.dsl.component
def model_eval():
    # Defining component configuration
    evaluation_component = kfp.dsl.ContainerOp(
        name='Model evaluation',
        image='gcr.io/ai-platform-01/train_image:latest',
        command=['python', 'evaluation.py'],
        file_outputs={"mlpipeline-metrics":'/mlpipeline-metrics.json'}
        )
    return evaluation_component

In [134]:

@kfp.dsl.pipeline(
  name="Bank churn pipeline",
  description="My first pipeline"
)
def final_pipeline():
    download_data = get_data()
    data_splitting = split().after(download_data)
    data_transform=data_transformation().after(data_splitting)
    training=model_train().after(data_transform)
    data_transformation_test=test_transformation().after(data_splitting,data_transform)
    model_evaluation=model_eval().after(data_transformation_test,training)

    

In [135]:
kfp.compiler.Compiler().compile(final_pipeline, 'final_pipeline.zip')

In [136]:
client = kfp.Client("d2c3bfe4c65f7b8-dot-us-central1.pipelines.googleusercontent.com")

In [137]:
EXPERIMENT_NAME = 'bank_churn'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

In [138]:
run = client.run_pipeline(experiment.id, 'Test1', 'final_pipeline.zip')